[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SkatAI/deeplearning/blob/master/notebooks/moliere_lstm_generation.ipynb)

# Génération de texte avec un LSTM : écrire comme Molière

**Objectif** : Entraîner un réseau LSTM character-level à générer du texte dans le style de Molière.

**Ce que vous allez apprendre** :
- Comment transformer du texte en séquences pour un réseau de neurones
- Comment fonctionne un LSTM pour la modélisation de séquences
- La différence entre prédiction de séries temporelles et génération de texte (spoiler : c'est le même principe !)
- Le rôle de la **temperature** dans la génération
- Le lien avec les LLMs modernes (GPT, Claude, Gemini...)

**Prérequis** : RNN, LSTM, GRU sur séries temporelles.

**Environnement** : Google Colab (free tier, CPU ou GPU T4)

---

## 1. L'intuition : des séries temporelles au texte

Vous avez déjà utilisé des LSTM pour prédire des séries temporelles. Le principe était :

> Étant donné une séquence de valeurs passées $[x_1, x_2, ..., x_t]$, prédire la prochaine valeur $x_{t+1}$

La génération de texte, c'est **exactement la même chose**, sauf qu'au lieu de prédire un nombre, on prédit le **prochain caractère** (ou mot) :

> Étant donné `"Quand l'Amour à vos yeux offre un choi"` → prédire `"x"`

C'est le principe fondamental derrière **tous** les modèles de langage actuels, y compris GPT-4 ou Claude. La différence ? L'échelle (milliards de paramètres, architecture Transformer), mais le concept de base est le même : **next token prediction**.

Dans ce TD, on va travailler au niveau **caractère** (char-level). Chaque "token" est une lettre, un espace, ou un signe de ponctuation. C'est plus simple qu'un tokenizer par mots ou sous-mots, et ça permet d'obtenir des résultats intéressants même avec un petit modèle.

## 2. Chargement des données

On utilise un corpus de pièces de Molière, disponible en CSV.

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Chargement du dataset
url = "https://skatai.com/data/df_moliere.csv"
df = pd.read_csv(url)

print(f"Nombre de répliques : {len(df)}")
print(f"Colonnes : {list(df.columns)}")
print(f"Pièces : {df['play_name'].nunique()}")
df.head(10)

In [ ]:
# On concatène toutes les répliques en un seul texte
# On ajoute un retour à la ligne entre chaque réplique pour garder la structure
text = "\n".join(df["cue"].dropna().astype(str).values)

print(f"Longueur du texte : {len(text):,} caractères")
print(f"\nExtrait :")
print(text[:500])

## 3. Tokenization character-level

En char-level, chaque caractère unique du corpus devient un "token". Notre vocabulaire est l'ensemble des caractères distincts.

Comparez avec les séries temporelles : là-bas vos données étaient déjà numériques. Ici, on doit d'abord **encoder** les caractères en nombres.

In [ ]:
from collections import Counter

# Construction du vocabulaire avec filtrage des caractères rares
# -------------------------------------------------------------
# MIN_CHAR_FREQ = 1  -> aucun filtrage (vocabulaire complet)
# MIN_CHAR_FREQ = 3+ -> on remplace les caractères très rares par <UNK>
MIN_CHAR_FREQ = 3
UNK_TOKEN = "<UNK>"

char_counts = Counter(text)
kept_char_list = sorted([ch for ch, n in char_counts.items() if n >= MIN_CHAR_FREQ])
removed_char_list = sorted([ch for ch, n in char_counts.items() if n < MIN_CHAR_FREQ])

chars = kept_char_list.copy()

# On garantit la présence d'un espace pour le padding de génération
if ' ' not in chars:
    chars.append(' ')
    chars = sorted(chars)

# Ajout du token inconnu en fin de vocabulaire
chars.append(UNK_TOKEN)

vocab_size = len(chars)
kept_chars = len(kept_char_list)
removed_chars = len(removed_char_list)

print(f"MIN_CHAR_FREQ : {MIN_CHAR_FREQ}")
print(f"Taille du vocabulaire : {vocab_size} tokens")
print(f"Caractères conservés ({kept_chars}) : {kept_char_list}")
print(f"Caractères rares remplacés par {UNK_TOKEN} ({removed_chars}) : {removed_char_list}")


In [ ]:
# Dictionnaires de correspondance caractère <-> index
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

unk_idx = char_to_idx[UNK_TOKEN]

# Encoder tout le texte en séquence d'entiers
text_encoded = np.array([char_to_idx.get(ch, unk_idx) for ch in text], dtype=np.int32)
unk_count = int(np.sum(text_encoded == unk_idx))

print(f"Texte original  : {text[:50]}")
print(f"Texte encodé    : {text_encoded[:50]}")
print(f"\nExemple : '{text[0]}' -> {text_encoded[0]} -> '{idx_to_char[text_encoded[0]]}'")
print(f"Nombre de tokens {UNK_TOKEN} dans le corpus : {unk_count}")


### Comparaison avec les séries temporelles

| | Séries temporelles | Texte (char-level) |
|---|---|---|
| **Token** | Valeur numérique (prix, température...) | Un caractère (lettre, espace, ponctuation) |
| **Vocabulaire** | Continu (infini) | Fini (~70-100 caractères) |
| **Encoding** | Pas nécessaire | char → index entier → embedding |
| **Prédiction** | Régression (valeur continue) | Classification (quel caractère parmi N ?) |
| **Loss** | MSE | Cross-entropy |

## 4. Création des séquences d'entraînement

Comme pour les séries temporelles, on crée des fenêtres glissantes. Pour chaque fenêtre de `SEQ_LENGTH` caractères, la cible est le caractère suivant.

### Pipeline `tf.data` pas à pas (avec échantillons)

On décompose volontairement la construction du dataset pour inspecter le contenu à chaque étape.


In [ ]:
import tensorflow as tf

# ============================================================
# >>> À EXPÉRIMENTER : changez SEQ_LENGTH et observez l'effet
# Valeurs suggérées : 40, 60, 100, 150
# ============================================================
SEQ_LENGTH = 40
BATCH_SIZE = 64
WINDOW_SHIFT = 5

# Étape 1: un dataset de scalaires (1 index de caractère par élément)
tokens_ds = tf.data.Dataset.from_tensor_slices(text_encoded)

print("Element spec tokens_ds:", tokens_ds.element_spec)
for token in tokens_ds.take(8):
    print(int(token.numpy()), end=' ')
print()

In [ ]:
# Étape 2: fenêtres glissantes de longueur SEQ_LENGTH + 1
windows_ds = tokens_ds.window(SEQ_LENGTH + 1, shift=WINDOW_SHIFT, drop_remainder=True)

# Un élément de windows_ds est lui-même un Dataset (fenêtre)
first_window_ds = next(iter(windows_ds.take(1)))
first_window_tokens = list(first_window_ds.as_numpy_iterator())

print("Type d'un élément de windows_ds:", type(first_window_ds))
print("Longueur première fenêtre:", len(first_window_tokens))
print("Premiers tokens fenêtre:", first_window_tokens[:10])

In [ ]:
# Étape 3: conversion de chaque fenêtre en tenseur [SEQ_LENGTH + 1]
sequences = windows_ds.flat_map(lambda w: w.batch(SEQ_LENGTH + 1))

print("Element spec sequences:", sequences.element_spec)
example_sequence = next(iter(sequences.take(1))).numpy()
print("Shape d'une séquence:", example_sequence.shape)
print("Premiers tokens séquence:", example_sequence[:10])


def split_input_target(sequence):
    """Sépare chaque séquence en input (tous sauf le dernier) et target (tous sauf le premier)."""
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

# Étape 4: mapping vers (input, target)
pairs_ds = sequences.map(split_input_target, num_parallel_calls=tf.data.AUTOTUNE)

print("Element spec pairs_ds:", pairs_ds.element_spec)
inp, tgt = next(iter(pairs_ds.take(1)))
print("Shape input:", inp.shape, "| Shape target:", tgt.shape)
print("Input (début):", inp.numpy()[:10])
print("Target(début):", tgt.numpy()[:10])

In [ ]:
# Étape 5: pipeline final pour l'entraînement
num_tokens = len(text_encoded)
num_sequences = max(0, (num_tokens - (SEQ_LENGTH + 1)) // WINDOW_SHIFT + 1)
steps_per_epoch = num_sequences // BATCH_SIZE  # drop_remainder=True

print(f"Nombre total de tokens       : {num_tokens}")
print(f"Nombre total de séquences    : {num_sequences}")
print(f"Batch size                   : {BATCH_SIZE}")
print(f"steps_per_epoch (estimé)     : {steps_per_epoch}")

if steps_per_epoch == 0:
    raise ValueError("steps_per_epoch=0: augmentez les données ou réduisez BATCH_SIZE/SEQ_LENGTH.")

dataset = (
    pairs_ds
    .shuffle(10000)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE)
)

# Vérification batch final
for input_example, target_example in dataset.take(1):
    print(f"Shape input  : {input_example.shape}  (batch, seq_length)")
    print(f"Shape target : {target_example.shape}")

**Remarquez** : le target est simplement l'input décalé d'une position. Pour chaque position $t$, le modèle doit prédire le caractère en position $t+1$. C'est exactement le même principe que le "sliding window" des séries temporelles.

> **Question pour vous** : que se passe-t-il si on augmente `SEQ_LENGTH` ? Quel est le compromis ?

## 5. Construction du modèle LSTM

L'architecture est simple :
1. **[Embedding](https://keras.io/api/layers/core_layers/embedding/)** : transforme chaque index de caractère en un vecteur dense (comme un word2vec, mais pour des caractères)
2. **LSTM** : apprend les dépendances séquentielles
3. **Dense + softmax** : prédit une distribution de probabilité sur les `vocab_size` caractères possibles

C'est un problème de **classification** à chaque pas de temps : parmi les ~80 caractères possibles, lequel vient ensuite ?


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras import mixed_precision

# Mixed precision pour accélérer l'entraînement GPU (ex: Colab T4)
mixed_precision.set_global_policy("mixed_float16")

# ============================================================
# >>> À EXPÉRIMENTER : modifiez ces hyperparamètres
# EMBEDDING_DIM : 64, 128, 256
# LSTM_UNITS : 128, 256, 512
# Ajoutez un 2e layer LSTM (return_sequences=True sur le premier)
# ============================================================
EMBEDDING_DIM = 64
LSTM_UNITS = 128

model = Sequential([
    # Embedding : chaque caractère (index) -> vecteur dense
    Embedding(vocab_size, EMBEDDING_DIM, input_length=SEQ_LENGTH),

    # LSTM : capture les dépendances dans la séquence
    LSTM(LSTM_UNITS, return_sequences=True),

    # Dropout pour la régularisation
    Dropout(0.2),

    # Couche de sortie : probabilité pour chaque caractère du vocabulaire
    Dense(vocab_size, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

### Comprendre l'architecture

- **Embedding(vocab_size, 64)** : Le modèle apprend une représentation vectorielle pour chaque caractère. Les caractères qui apparaissent dans des contextes similaires (ex: voyelles entre elles) auront des embeddings proches.
- **LSTM(128)** : La couche LSTM lit la séquence caractère par caractère et maintient un état interne (cell state + hidden state). C'est cet état qui "résume" ce qui a été lu et permet de prédire la suite.
- **Dense(vocab_size, softmax)** : La sortie est un vecteur de probabilités. Si le vocabulaire a 80 caractères, on obtient 80 probabilités qui somment à 1.

**Pourquoi `sparse_categorical_crossentropy` et pas `categorical_crossentropy` ?**

- Avec `sparse_categorical_crossentropy`, les labels restent des entiers (`y = 17`, `y = 42`, ...).
- Avec `categorical_crossentropy`, il faudrait convertir chaque label en one-hot de taille `vocab_size`.
- Ici nos targets sont des index entiers de caractères (shape `(batch, seq_length)`), donc `sparse_categorical_crossentropy` est la forme naturelle, plus simple et plus économe en mémoire.


## 6. Entraînement

On entraîne sur quelques epochs. Sur free Colab (T4 GPU ou CPU), ça devrait prendre quelques minutes par epoch.

**Pourquoi Colab affiche parfois `xx/Unknown` ?**

Avec certains pipelines `tf.data` (notamment `window` + `flat_map`), TensorFlow ne connaît pas toujours la cardinalité exacte du dataset au moment de l'affichage de la barre de progression.

Ici, on calcule explicitement `steps_per_epoch` dans la section 4, puis on le passe à `model.fit(...)` pour obtenir une progression `current/total`.

**Astuce** : Activez le GPU dans Colab via `Runtime > Change runtime type > T4 GPU` pour accélérer l'entraînement.


In [ ]:
# ============================================================
# >>> À EXPÉRIMENTER : nombre d'epochs
# 5 epochs  : le modèle commence à apprendre la structure
# 15 epochs : résultats intéressants
# 30+ epochs : meilleure qualité (mais plus long)
# ============================================================
EPOCHS = 15

history = model.fit(
    dataset,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    verbose=1
)


In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(history.history['loss'])
ax1.set_title('Loss par epoch')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.grid(True, alpha=0.3)

ax2.plot(history.history['accuracy'])
ax2.set_title('Accuracy par epoch')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nLoss finale : {history.history['loss'][-1]:.4f}")
print(f"Accuracy finale : {history.history['accuracy'][-1]:.4f}")

### Sauvegarder, télécharger et recharger un modèle

Après un entraînement court (ex: 3-5 epochs), vous pouvez télécharger ce modèle puis charger un modèle pré-entraîné plus long pour comparer la génération.


In [ ]:
# Modèle entraîné dans ce notebook (entraînement court)
quick_model = model
pretrained_model = None
inference_model = quick_model  # modèle utilisé par défaut pour la génération

MODEL_SAVE_PATH = "moliere_lstm_quick.keras"
quick_model.save(MODEL_SAVE_PATH)
print(f"Modèle sauvegardé: {MODEL_SAVE_PATH}")

# Téléchargement (Colab)
try:
    from google.colab import files
    files.download(MODEL_SAVE_PATH)
except Exception:
    print("Téléchargement auto indisponible hors Colab. Le fichier est sauvegardé localement.")


In [ ]:
# Optionnel: charger un modèle pré-entraîné depuis un fichier local (upload Colab)
# -------------------------------------------------------------------------------
# from google.colab import files
# uploaded = files.upload()  # choisissez un fichier .keras
# uploaded_path = next(iter(uploaded.keys()))
#
# pretrained_model = tf.keras.models.load_model(uploaded_path)
# inference_model = pretrained_model
# print(f"Modèle pré-entraîné chargé depuis upload: {uploaded_path}")


In [ ]:
# Optionnel: charger un modèle pré-entraîné depuis une URL
# -------------------------------------------------------
# PRETRAINED_MODEL_URL = "https://.../moliere_lstm_long.keras"
#
# local_pretrained_path = tf.keras.utils.get_file(
#     fname="moliere_lstm_pretrained.keras",
#     origin=PRETRAINED_MODEL_URL
# )
# pretrained_model = tf.keras.models.load_model(local_pretrained_path)
# inference_model = pretrained_model
# print(f"Modèle pré-entraîné chargé depuis URL: {local_pretrained_path}")


In [ ]:
# Optionnel: comparaison rapide entre modèle court et modèle pré-entraîné
# Décommentez après avoir chargé pretrained_model
# ---------------------------------------------------------------
# seed_text_compare = "Quand l'Amour"
# temperature_compare = 0.8
#
# if pretrained_model is None:
#     print("Chargez d'abord un modèle pré-entraîné (upload ou URL).")
# else:
#     print("--- Modèle entraîné rapidement (quick_model) ---")
#     print(generate_text(quick_model, seed_text_compare, num_generate=200, temperature=temperature_compare))
#     print()
#     print("--- Modèle pré-entraîné (long training) ---")
#     print(generate_text(pretrained_model, seed_text_compare, num_generate=200, temperature=temperature_compare))


## 7. Génération de texte et temperature

Maintenant, on utilise le modèle pour **générer** du texte. Le processus est autorégressif :

1. On donne une séquence de départ (seed)
2. Le modèle prédit une distribution de probabilité sur le prochain caractère
3. On **échantillonne** un caractère selon cette distribution
4. On ajoute ce caractère à la séquence et on recommence

### Le rôle de la temperature

La **temperature** contrôle la "créativité" de la génération en modifiant la distribution de probabilité avant l'échantillonnage :

$$p_i = \frac{\exp(\log(p_i) / T)}{\sum_j \exp(\log(p_j) / T)}$$

- **T → 0** : le modèle choisit presque toujours le caractère le plus probable → texte répétitif, "sûr"
- **T = 1** : distribution non modifiée → équilibre
- **T > 1** : distribution plus uniforme → texte plus "créatif", plus de surprises, mais aussi plus d'erreurs

C'est exactement le même paramètre que vous retrouvez dans les API de GPT, Claude, etc.

In [ ]:
def generate_text(model, start_string, num_generate=300, temperature=1.0):
    """
    Génère du texte caractère par caractère.

    Args:
        model: le modèle LSTM entraîné
        start_string: texte de départ (seed)
        num_generate: nombre de caractères à générer
        temperature: contrôle la créativité (0.2 = conservateur, 1.5 = créatif)
    """
    unk_idx = char_to_idx[UNK_TOKEN]
    pad_idx = char_to_idx.get(' ', unk_idx)

    # Encoder le texte de départ (caractères inconnus -> <UNK>)
    input_eval = [char_to_idx.get(ch, unk_idx) for ch in start_string]

    # S'assurer que la séquence fait exactement SEQ_LENGTH
    if len(input_eval) < SEQ_LENGTH:
        # Padding à gauche avec des espaces
        pad = [pad_idx] * (SEQ_LENGTH - len(input_eval))
        input_eval = pad + input_eval
    else:
        input_eval = input_eval[-SEQ_LENGTH:]

    generated = list(start_string)

    for _ in range(num_generate):
        input_array = np.array([input_eval])

        # Prédiction : distribution de probabilité sur le vocabulaire
        predictions = model.predict(input_array, verbose=0)[0, -1]

        # Application de la temperature
        predictions = np.log(predictions + 1e-8) / temperature
        predictions = np.exp(predictions)
        predictions = predictions / np.sum(predictions)

        # Échantillonnage selon la distribution
        predicted_id = np.random.choice(len(predictions), p=predictions)

        # Ajouter le caractère généré et décaler la fenêtre
        next_token = idx_to_char[predicted_id]
        generated.append(next_token if next_token != UNK_TOKEN else '?')
        input_eval = input_eval[1:] + [predicted_id]

    return ''.join(generated)


In [ ]:
# ============================================================
# >>> À EXPÉRIMENTER : changez la temperature et le seed
# ============================================================

seed_text = "Quand l'Amour"

print("=" * 70)
for temp in [0.2, 0.5, 0.8, 1.0, 1.3]:
    print(f"\n--- Temperature = {temp} ---\n")
    result = generate_text(inference_model, seed_text, num_generate=200, temperature=temp)
    print(result)
    print()
print("=" * 70)

### Observations

> **Exercice** : Comparez les textes générés aux différentes temperatures.
>
> 1. À temperature basse (0.2), que remarquez-vous ? Le texte est-il varié ?
> 2. À temperature haute (1.3), la qualité diminue-t-elle ? Pourquoi ?
> 3. Quelle temperature vous semble produire le meilleur compromis ?
>
> **Essayez aussi** avec différents textes de départ (seed). Le modèle génère-t-il différemment selon le contexte initial ?

## 8. À vous de jouer !

### Expérience 1 : Impact de la longueur de séquence

Revenez à la section 4 et changez `SEQ_LENGTH`. Essayez 40, 100, et 150. Réentraînez et comparez la qualité du texte généré.

> Plus la séquence est longue, plus le modèle a de contexte, mais l'entraînement est plus lent et nécessite plus de mémoire.

### Expérience 2 : Architecture du modèle

Modifiez le modèle dans la section 5 :
- Essayez un modèle plus petit (LSTM 128 units) vs plus grand (LSTM 512)
- Ajoutez une deuxième couche LSTM :

```python
model = Sequential([
    Embedding(vocab_size, EMBEDDING_DIM, input_length=SEQ_LENGTH),
    LSTM(256, return_sequences=True),   # return_sequences=True pour empiler
    LSTM(256, return_sequences=False),
    Dropout(0.2),
    Dense(vocab_size, activation='softmax')
])
```

- Remplacez le LSTM par un GRU. Le résultat change-t-il ?

### Expérience 3 : Sous-corpus

Essayez d'entraîner sur une seule pièce. Le style est-il différent ?

In [ ]:
# Exemple : entraîner sur une seule pièce
# Décommentez et adaptez :

# piece = "tartuffe"  # ou "misanthrope", etc.
# df_piece = df[df['play_name'] == piece]
# text_piece = "\n".join(df_piece["cue"].dropna().astype(str).values)
# print(f"Pièce : {piece}, {len(text_piece):,} caractères")
# print(f"Pièces disponibles : {sorted(df['play_name'].unique())}")

## 9. Bonus : comparaison avec un modèle Hugging Face

Pour mettre en perspective, comparons notre petit LSTM avec un modèle de langage pré-entraîné. On utilise un modèle GPT-2 (le plus petit, 124M paramètres) pour générer du texte à partir du même seed.

Notre LSTM a quelques centaines de milliers de paramètres et a été entraîné sur quelques centaines de Ko de texte. GPT-2 "small" a 124 millions de paramètres et a été entraîné sur des milliards de mots.

In [ ]:
# Installation si nécessaire (déjà installé sur Colab en général)
# !pip install transformers -q

from transformers import pipeline, set_seed

seed_text_hf = "Quand l'Amour"
set_seed(42)

# 1) GPT-2 (baseline)
try:
    generator_gpt2 = pipeline('text-generation', model='gpt2', device=-1)  # CPU

    result_gpt2 = generator_gpt2(
        seed_text_hf,
        max_length=150,
        num_return_sequences=1,
        temperature=0.8,
        do_sample=True
    )

    print("--- GPT-2 (124M params, entraîné majoritairement en anglais) ---")
    print(result_gpt2[0]['generated_text'])
    print()
except Exception as e:
    print(f"Erreur lors du chargement de GPT-2 : {e}")
    print()

# 2) Modèle français: asi/gpt-fr-cased-small
try:
    generator_gpt_fr = pipeline('text-generation', model='asi/gpt-fr-cased-small', device=-1)  # CPU

    result_gpt_fr = generator_gpt_fr(
        seed_text_hf,
        max_length=150,
        num_return_sequences=1,
        temperature=0.8,
        do_sample=True
    )

    print("--- asi/gpt-fr-cased-small (modèle français) ---")
    print(result_gpt_fr[0]['generated_text'])
    print()
except Exception as e:
    print(f"Erreur lors du chargement de asi/gpt-fr-cased-small : {e}")
    print()

# 3) Notre LSTM
print("--- Notre LSTM (entraîné sur Molière uniquement) ---")
print(generate_text(inference_model, seed_text_hf, num_generate=150, temperature=0.8))


### Discussion

> **Question** : GPT-2 est entraîné principalement en anglais. Comment se comporte-t-il sur un prompt en français ? Pourquoi ?
>
> **Question** : Notre LSTM produit du texte "à la Molière" car il n'a vu que ça. C'est à la fois sa force (spécialisation) et sa limite (pas de connaissance générale). Comment les LLMs modernes gèrent-ils ce compromis ?
>
> **Pour aller plus loin** : essayez avec un modèle comme `"asi/gpt-fr-cased-small"` (GPT-2 entraîné sur du français) si disponible.

## 10. Récapitulatif et liens avec les LLMs

### Ce qu'on a fait
1. **Tokenization** : transformer du texte en séquence d'entiers (ici char-level)
2. **Séquences** : créer des fenêtres glissantes input/target (comme en séries temporelles)
3. **Modèle** : Embedding → LSTM → Dense (classification sur le vocabulaire)
4. **Génération** : prédiction autoregressive + sampling avec temperature

### Le pont vers les LLMs modernes

| | Notre LSTM | GPT / Claude / Gemini |
|---|---|---|
| **Tokenization** | Character-level (~80 tokens) | Subword (BPE, ~50K-100K tokens) |
| **Architecture** | LSTM | Transformer (attention) |
| **Contexte** | 60-100 caractères | 8K - 1M+ tokens |
| **Paramètres** | ~300K | 7B - 1000B+ |
| **Entraînement** | Un corpus (~500 Ko) | Internet (~To de texte) |
| **Principe** | Next token prediction | Next token prediction |

Le principe fondamental est **identique** : prédire le prochain token. Ce qui change, c'est l'échelle, l'architecture (attention vs récurrence), et la quantité de données.

### Pour aller plus loin
- **Transformers** : comprendre le mécanisme d'attention et pourquoi il a remplacé les LSTM pour le langage
- **Tokenization BPE** : comment les LLMs découpent le texte en sous-mots
- **Fine-tuning** : adapter un LLM pré-entraîné à un domaine spécifique (ex: textes juridiques, style littéraire)
- **RLHF** : comment les LLMs sont alignés avec les préférences humaines après le pré-entraînement